# Filtering results

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


## Midtre Lovenbreen
### Load all stations

In [ ]:
sw110 = xr.open_dataset('data_qc/aws-sw-110-20210426-20220401.nc')
sw120 = xr.open_dataset('data_qc/aws-sw-120-20210426-20220330.nc')
sw125 = xr.open_dataset('data_qc/aws-sw-125-20210427-20220401.nc')
sw130 = xr.open_dataset('data_qc/aws-sw-130-20210426-20220401.nc')
sw140 = xr.open_dataset('data_qc/aws-sw-140-20210426-20220429.nc')
#swcorbel = xr.open_dataset('data_qc/aws-sw-corbel-20210426-20220429.nc')


plt.figure()
sw110.snd.plot(label="sw110")
sw120.snd.plot(label="sw120")
#sw125.snd.plot(label="sw125") # failed - no data
sw130.snd.plot(label="sw130")
sw140.snd.plot(label="sw140")
plt.legend()
plt.show()

In [ ]:
sw140

In [ ]:
plt.figure()
plt.scatter(sw110.time, sw110.snd,label="sw110", alpha=.5, s=.1)
plt.scatter(sw120.time, sw120.snd,label="sw120", alpha=.5, s=.1)
#plt.scatter(sw125.time, sw125.snd,label="sw125", alpha=.5, s=.1) # failed - no data
plt.scatter(sw130.time, sw130.snd,label="sw130", alpha=.5, s=.1)
plt.scatter(sw140.time, sw140.snd,label="sw140", alpha=.5, s=.1)
plt.legend()
plt.show()

### Snow depth

In [ ]:
plt.figure()
sw110.snd.plot(label="sw110")
sw120.snd.plot(label="sw120")
#sw125.snd.plot(label="sw125") # failed - no data
sw130.snd.plot(label="sw130")
sw140.snd.plot(label="sw140")
plt.legend()
plt.show()

### Air temperature

In [ ]:
plt.figure()
sw110.ta.plot(label="sw110")
sw120.ta.plot(label="sw120")
sw125.ta.plot(label="sw125")
sw130.ta.plot(label="sw130")
sw140.ta.plot(label="sw140")
plt.legend()
plt.show()

### Surface temperature

In [ ]:
plt.figure()
sw110.ts.plot(label="sw110")
sw120.ts.plot(label="sw120")
sw125.ts.plot(label="sw125")
sw130.ts.plot(label="sw130")
sw140.ts.plot(label="sw140")
plt.legend()
plt.show()

### Relative humidity

In [ ]:
plt.figure()
sw110.hur.plot(label="sw110")
sw120.hur.plot(label="sw120")
#sw125.hur.plot(label="sw125") # failed - no data
sw130.hur.plot(label="sw130")
sw140.hur.plot(label="sw140")
plt.legend()
plt.show()

### Air pressure

In [ ]:
plt.figure()
plt.scatter(sw110.time, sw110.ps,label="sw110", alpha=.5, s=.1)
plt.scatter(sw120.time, sw120.ps,label="sw120", alpha=.5, s=.1)
#plt.scatter(sw125.time, sw125.ps,label="sw125", alpha=.5, s=.1) # failed - no data
plt.scatter(sw130.time, sw130.ps,label="sw130", alpha=.5, s=.1)
plt.scatter(sw140.time, sw140.ps,label="sw140", alpha=.5, s=.1)
plt.legend()
plt.show()

In [ ]:
plt.figure()
sw110.ps.plot(label="sw110")
sw120.ps.plot(label="sw120")
#sw125.ps.plot(label="sw125") # failed - no data
sw130.ps.plot(label="sw130")
sw140.ps.plot(label="sw140")
plt.legend()
plt.show()

### Wind speed and direction

No wind sensor for the following wsn weather stations: sw125 and sw130

In [ ]:
plt.figure()
sw110.ws.plot(label="sw110")
sw120.ws.plot(label="sw120")
sw140.ws.plot(label="sw140")
plt.legend()
plt.show()

In [ ]:
plt.figure()
sw110.dw.plot(label="sw110")
sw120.dw.plot(label="sw120")
sw140.dw.plot(label="sw140")
plt.legend()
plt.show()

### Load reference data

In [ ]:
sw110 = xr.open_dataset('data_qc/reference/aws-sw-110-20210426-20220401.nc')
sw120 = xr.open_dataset('data_qc/reference/aws-sw-120-20210426-20220330.nc')
sw125 = xr.open_dataset('data_qc/reference/aws-sw-125-20210427-20220401.nc')
sw130 = xr.open_dataset('data_qc/reference/aws-sw-130-20210426-20220401.nc')
sw140 = xr.open_dataset('data_qc/reference/aws-sw-140-20210426-20220501.nc')
swcorbel = xr.open_dataset('data_qc/reference/aws-sw-corbel-20210426-20220429.nc')
sw140

plt.figure()
sw110.snd.plot(label="sw110")
sw120.snd.plot(label="sw120")
#sw125.snd.plot(label="sw125") # failed - no data
sw130.snd.plot(label="sw130")
sw140.snd.plot(label="sw140")
plt.legend()
plt.show()

---
---

## Kongsvegen

In [5]:
sw200 = xr.open_dataset('data_qc/aws-sw-200-20210501-20220503.nc')
sw205 = xr.open_dataset('data_qc/aws-sw-205-20210501-20220501.nc')
sw210 = xr.open_dataset('data_qc/aws-sw-210-20210429-20220501.nc')
#sw215 = xr.open_dataset('data_qc/aws-sw-001-20210429-20220501.nc') #missing file
sw220 = xr.open_dataset('data_qc/aws-sw-220-20210429-20220501.nc')
sw225 = xr.open_dataset('data_qc/aws-sw-225-20210501-20220501.nc')
sw230 = xr.open_dataset('data_qc/aws-sw-230-20210429-20220501.nc')
sw235 = xr.open_dataset('data_qc/aws-sw-235-20210429-20220501.nc')
sw240 = xr.open_dataset('data_qc/aws-sw-240-20210429-20220501.nc')
sw250 = xr.open_dataset('data_qc/aws-sw-250-20210501-20220501.nc')
sw240

<xarray.Dataset>
Dimensions:    (time: 49897, station: 1)
Coordinates:
  * time       (time) datetime64[ns] 2021-04-29T10:47:58.000488281 ... 2022-0...
  * station    (station) |S16 b'sw-240'
Data variables:
    orog       (station) float32 ...
    latitude   (station) float32 ...
    longitude  (station) float32 ...
    slope      (station) float32 ...
    azimuth    (station) float32 ...
    dw         (time, station) float32 ...
    snd        (time, station) float32 ...
    ps         (time, station) float32 ...
    hur        (time, station) float32 ...
    ta         (time, station) float32 ...
    ts         (time, station) float32 ...
    ws         (time, station) float32 ...
Attributes: (12/38)
    date_created:              2022-06-17
    creator_name:              Simon Filhol, Pierre Marie Lefeuvre, Jean-Char...
    creator_email:             simon.filhol@geo.uio.no, pierre.lefeuvre@npola...
    creator_institution:       University of Oslo, Norwegian Polar Institute,...
    creator_url:               www.mn.uio.no/geo/english/people/aca/geohyd/si...
    creator_type:              person, person, person
    ...                        ...
    geospatial_lat_max:        78.803
    geospatial_lon_min:        12.9573
    geospatial_lon_max:        12.9573
    time_coverage_start:       2021-04-29T10:47:58+00:00
    time_coverage_end:         2022-04-30T21:47:58+00:00
    time_coverage_resolution:  P635S

## Snow depth

In [25]:
plt.figure()
sw200.snd.plot(label="sw200")
sw205.snd.plot(label="sw205")
sw210.snd.plot(label="sw210")
sw215.snd.plot(label="sw215")
sw220.snd.plot(label="sw220")
sw225.snd.plot(label="sw225")
sw230.snd.plot(label="sw230")
#sw235.snd.plot(label="sw235") # failed - no data
sw240.snd.plot(label="sw240")
sw250.snd.plot(label="sw250")
plt.legend()
plt.show()

In [26]:
plt.figure()
plt.scatter(sw200.time, sw200.snd,label="sw200", alpha=.5, s=.1)
plt.scatter(sw205.time, sw205.snd,label="sw205", alpha=.5, s=.1)
plt.scatter(sw210.time, sw210.snd,label="sw210", alpha=.5, s=.1)
plt.scatter(sw215.time, sw215.snd,label="sw215", alpha=.5, s=.1)
plt.scatter(sw220.time, sw220.snd,label="sw220", alpha=.5, s=.1)
plt.scatter(sw225.time, sw225.snd,label="sw225", alpha=.5, s=.1)
plt.scatter(sw230.time, sw230.snd,label="sw230", alpha=.5, s=.1)
#plt.scatter(sw235.time, sw235.snd,label="sw235", alpha=.5, s=.1) # failed - no data
plt.scatter(sw240.time, sw240.snd,label="sw240", alpha=.5, s=.1)
plt.scatter(sw250.time, sw250.snd,label="sw250", alpha=.5, s=.1)
plt.legend()
plt.show()

## Air temperature

In [24]:
plt.figure()
plt.scatter(sw200.time, sw200.ta,label="sw200", alpha=.5, s=.1)
plt.scatter(sw205.time, sw205.ta,label="sw205", alpha=.5, s=.1)
plt.scatter(sw210.time, sw210.ta,label="sw210", alpha=.5, s=.1)
plt.scatter(sw215.time, sw215.ta,label="sw215", alpha=.5, s=.1)
plt.scatter(sw220.time, sw220.ta,label="sw220", alpha=.5, s=.1)
plt.scatter(sw225.time, sw225.ta,label="sw225", alpha=.5, s=.1)
plt.scatter(sw230.time, sw230.ta,label="sw230", alpha=.5, s=.1)
plt.scatter(sw235.time, sw235.ta,label="sw235", alpha=.5, s=.1)
plt.scatter(sw240.time, sw240.ta,label="sw240", alpha=.5, s=.1)
plt.scatter(sw250.time, sw250.ta,label="sw250", alpha=.5, s=.1)
plt.legend()
plt.show()

In [18]:
plt.figure()
sw230.snd.dropna(dim='time').plot(label="sw230")
plt.show()

In [19]:
plt.figure()
sw240.snd.dropna(dim='time').plot(label="sw230")
plt.show()

In [23]:
plt.figure()
sw210.snd.dropna(dim='time').plot(label="sw230")
plt.show()

## Etonbreen

In [ ]:
#eton = xr.open_dataset('data_qc/aws-eton-2-20040421-20210914.nc')
plt.figure()
eton.snd.plot()
plt.figure()
eton.ps.plot()
plt.figure()
eton.ps.plot()